##**2. Word2Vec**
1. 주어진 단어들을 word2vec 모델에 들어갈 수 있는 형태로 만든다
2. CBOW, Skip-gram 모델을 각각 구현한다.

### **필요 패키지 import**

In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 460kB 50.7MB/s 
     |████████████████████████████████| 92kB 11.7MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [2]:
from tqdm import tqdm
from konlpy.tag import Okt
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict

import torch
import copy
import numpy as np

### **데이터 전처리**

In [3]:
train_data = [
  "정말 맛있습니다. 추천합니다.",
  "기대했던 것보단 별로였네요.",
  "다 좋은데 가격이 너무 비싸서 다시 가고 싶다는 생각이 안 드네요.",
  "완전 최고입니다! 재방문 의사 있습니다.",
  "음식도 서비스도 다 만족스러웠습니다.",
  "위생 상태가 좀 별로였습니다. 좀 더 개선되기를 바랍니다.",
  "맛도 좋았고 직원분들 서비스도 너무 친절했습니다.",
  "기념일에 방문했는데 음식도 분위기도 서비스도 다 좋았습니다.",
  "전반적으로 음식이 너무 짰습니다. 저는 별로였네요.",
  "위생에 조금 더 신경 썼으면 좋겠습니다. 조금 불쾌했습니다."       
]

test_words = ["음식", "맛", "서비스", "위생", "가격"]

In [4]:
tokenizer = Okt()

In [5]:
def make_tokenized(data):
  tokenized = []
  for sent in tqdm(data):
    tokens = tokenizer.morphs(sent, stem=True)
    tokenized.append(tokens)

  return tokenized

In [6]:
train_tokenized = make_tokenized(train_data)

100%|██████████| 10/10 [00:05<00:00,  1.82it/s]


In [7]:
word_count = defaultdict(int)

for tokens in tqdm(train_tokenized):
  for token in tokens:
    word_count[token] += 1

100%|██████████| 10/10 [00:00<00:00, 19972.88it/s]


In [8]:
word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)
print(list(word_count))

[('.', 14), ('도', 7), ('이다', 4), ('좋다', 4), ('별로', 3), ('다', 3), ('이', 3), ('너무', 3), ('음식', 3), ('서비스', 3), ('하다', 2), ('방문', 2), ('위생', 2), ('좀', 2), ('더', 2), ('에', 2), ('조금', 2), ('정말', 1), ('맛있다', 1), ('추천', 1), ('기대하다', 1), ('것', 1), ('보단', 1), ('가격', 1), ('비싸다', 1), ('다시', 1), ('가다', 1), ('싶다', 1), ('생각', 1), ('안', 1), ('드네', 1), ('요', 1), ('완전', 1), ('최고', 1), ('!', 1), ('재', 1), ('의사', 1), ('있다', 1), ('만족스럽다', 1), ('상태', 1), ('가', 1), ('개선', 1), ('되다', 1), ('기르다', 1), ('바라다', 1), ('맛', 1), ('직원', 1), ('분들', 1), ('친절하다', 1), ('기념일', 1), ('분위기', 1), ('전반', 1), ('적', 1), ('으로', 1), ('짜다', 1), ('저', 1), ('는', 1), ('신경', 1), ('써다', 1), ('불쾌하다', 1)]


In [9]:
w2i = {}
for pair in tqdm(word_count):
  if pair[0] not in w2i:
    w2i[pair[0]] = len(w2i)

100%|██████████| 60/60 [00:00<00:00, 40813.86it/s]


In [10]:
print(train_tokenized)
print(w2i)

[['정말', '맛있다', '.', '추천', '하다', '.'], ['기대하다', '것', '보단', '별로', '이다', '.'], ['다', '좋다', '가격', '이', '너무', '비싸다', '다시', '가다', '싶다', '생각', '이', '안', '드네', '요', '.'], ['완전', '최고', '이다', '!', '재', '방문', '의사', '있다', '.'], ['음식', '도', '서비스', '도', '다', '만족스럽다', '.'], ['위생', '상태', '가', '좀', '별로', '이다', '.', '좀', '더', '개선', '되다', '기르다', '바라다', '.'], ['맛', '도', '좋다', '직원', '분들', '서비스', '도', '너무', '친절하다', '.'], ['기념일', '에', '방문', '하다', '음식', '도', '분위기', '도', '서비스', '도', '다', '좋다', '.'], ['전반', '적', '으로', '음식', '이', '너무', '짜다', '.', '저', '는', '별로', '이다', '.'], ['위생', '에', '조금', '더', '신경', '써다', '좋다', '.', '조금', '불쾌하다', '.']]
{'.': 0, '도': 1, '이다': 2, '좋다': 3, '별로': 4, '다': 5, '이': 6, '너무': 7, '음식': 8, '서비스': 9, '하다': 10, '방문': 11, '위생': 12, '좀': 13, '더': 14, '에': 15, '조금': 16, '정말': 17, '맛있다': 18, '추천': 19, '기대하다': 20, '것': 21, '보단': 22, '가격': 23, '비싸다': 24, '다시': 25, '가다': 26, '싶다': 27, '생각': 28, '안': 29, '드네': 30, '요': 31, '완전': 32, '최고': 33, '!': 34, '재': 35, '의사': 36, '있다': 37, '만족스럽다': 38, '상태

In [11]:
class CBOWDataset(Dataset):
  def __init__(self, train_tokenized, window_size=2):
    self.x = []
    self.y = []

    for tokens in tqdm(train_tokenized):
      token_ids = [w2i[token] for token in tokens]
      for i, id in enumerate(token_ids):
        if i-window_size >= 0 and i+window_size < len(token_ids):
          self.x.append(token_ids[i-window_size:i] + token_ids[i+1:i+window_size+1])
          self.y.append(id)

    self.x = torch.LongTensor(self.x)  # (전체 데이터 개수, 2 * window_size)
    self.y = torch.LongTensor(self.y)  # (전체 데이터 개수)

  def __len__(self):
    return self.x.shape[0]

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

In [12]:
class SkipGramDataset(Dataset):
  def __init__(self, train_tokenized, window_size=2):
    self.x = []
    self.y = []

    for tokens in tqdm(train_tokenized):
      token_ids = [w2i[token] for token in tokens]
      for i, id in enumerate(token_ids):
        if i-window_size >= 0 and i+window_size < len(token_ids):
          self.y += (token_ids[i-window_size:i] + token_ids[i+1:i+window_size+1])
          self.x += [id] * 2 * window_size

    self.x = torch.LongTensor(self.x)  # (전체 데이터 개수)
    self.y = torch.LongTensor(self.y)  # (전체 데이터 개수)

  def __len__(self):
    return self.x.shape[0]

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

In [13]:
cbow_set = CBOWDataset(train_tokenized)
skipgram_set = SkipGramDataset(train_tokenized)
print(list(skipgram_set))

100%|██████████| 10/10 [00:00<00:00, 6136.51it/s]

[(tensor(0), tensor(17)), (tensor(0), tensor(18)), (tensor(0), tensor(19)), (tensor(0), tensor(10)), (tensor(19), tensor(18)), (tensor(19), tensor(0)), (tensor(19), tensor(10)), (tensor(19), tensor(0)), (tensor(22), tensor(20)), (tensor(22), tensor(21)), (tensor(22), tensor(4)), (tensor(22), tensor(2)), (tensor(4), tensor(21)), (tensor(4), tensor(22)), (tensor(4), tensor(2)), (tensor(4), tensor(0)), (tensor(23), tensor(5)), (tensor(23), tensor(3)), (tensor(23), tensor(6)), (tensor(23), tensor(7)), (tensor(6), tensor(3)), (tensor(6), tensor(23)), (tensor(6), tensor(7)), (tensor(6), tensor(24)), (tensor(7), tensor(23)), (tensor(7), tensor(6)), (tensor(7), tensor(24)), (tensor(7), tensor(25)), (tensor(24), tensor(6)), (tensor(24), tensor(7)), (tensor(24), tensor(25)), (tensor(24), tensor(26)), (tensor(25), tensor(7)), (tensor(25), tensor(24)), (tensor(25), tensor(26)), (tensor(25), tensor(27)), (tensor(26), tensor(24)), (tensor(26), tensor(25)), (tensor(26), tensor(27)), (tensor(26), tens

### **모델 Class 구현**




*   `self.embedding`: `vocab_size` 크기의 one-hot vector를 특정 크기의 `dim` 차원으로 embedding 시키는 layer.
*   `self.linear`: 변환된 embedding vector를 다시 원래 `vocab_size`로 바꾸는 layer.


In [14]:
class CBOW(nn.Module):
  def __init__(self, vocab_size, dim):
    super(CBOW, self).__init__()
    self.embedding = nn.Embedding(vocab_size, dim, sparse=True)
    self.linear = nn.Linear(dim, vocab_size)

  # B: batch size, W: window size, d_w: word embedding size, V: vocab size
  def forward(self, x):  # x: (B, 2W)
    embeddings = self.embedding(x)  # (B, 2W, d_w)
    embeddings = torch.sum(embeddings, dim=1)  # (B, d_w)
    output = self.linear(embeddings)  # (B, V)
    return output

In [15]:
class SkipGram(nn.Module):
  def __init__(self, vocab_size, dim):
    super(SkipGram, self).__init__()
    self.embedding = nn.Embedding(vocab_size, dim, sparse=True)
    self.linear = nn.Linear(dim, vocab_size)

  # B: batch size, W: window size, d_w: word embedding size, V: vocab size
  def forward(self, x): # x: (B)
    embeddings = self.embedding(x)  # (B, d_w)
    output = self.linear(embeddings)  # (B, V)
    return output

In [16]:
cbow = CBOW(vocab_size=len(w2i), dim=256)
skipgram = SkipGram(vocab_size=len(w2i), dim=256)

### **모델 학습**

In [17]:
batch_size=4
learning_rate = 5e-4
num_epochs = 5
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

cbow_loader = DataLoader(cbow_set, batch_size=batch_size)
skipgram_loader = DataLoader(skipgram_set, batch_size=batch_size)

In [18]:
cbow.train()
cbow = cbow.to(device)
optim = torch.optim.SGD(cbow.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

for e in range(1, num_epochs+1):
  print("#" * 50)
  print(f"Epoch: {e}")
  for batch in tqdm(cbow_loader):
    x, y = batch
    x, y = x.to(device), y.to(device) # (B, W), (B)
    output = cbow(x)  # (B, V)
 
    optim.zero_grad()
    loss = loss_function(output, y)
    loss.backward()
    optim.step()

    print(f"Train loss: {loss.item()}")

print("Finished.")

  6%|▋         | 1/16 [00:00<00:02,  5.25it/s]

##################################################
Epoch: 1
Train loss: 4.777403831481934
Train loss: 3.4267449378967285
Train loss: 5.699966907501221
Train loss: 6.134458541870117
Train loss: 5.373473167419434


100%|██████████| 16/16 [00:00<00:00, 686.81it/s]

Train loss: 4.801996231079102
Train loss: 5.780697822570801
Train loss: 4.974893569946289
Train loss: 4.040414333343506
Train loss: 4.373447418212891
Train loss: 4.411188125610352
Train loss: 4.812112808227539
Train loss: 5.001648426055908
Train loss: 4.971104621887207
Train loss: 5.1489996910095215
Train loss: 4.694040298461914
##################################################
Epoch: 2
Train loss: 4.59750509262085
Train loss: 3.3064732551574707
Train loss: 5.552374839782715
Train loss: 5.982983112335205
Train loss: 5.233875274658203
Train loss: 4.522367000579834
Train loss: 5.582163333892822
Train loss: 4.844361782073975
Train loss: 3.9280333518981934
Train loss: 4.188053607940674
Train loss: 4.242216110229492
Train loss: 4.422765731811523
Train loss: 4.859151363372803
Train loss: 4.833887577056885
Train loss: 4.993827819824219
Train loss: 4.563860893249512
##################################################
Epoch: 3
Train loss: 4.4205756187438965
Train loss: 3.1889092922210693
Train 

In [19]:
skipgram.train()
skipgram = skipgram.to(device)
optim = torch.optim.SGD(skipgram.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

for e in range(1, num_epochs+1):
  print("#" * 50)
  print(f"Epoch: {e}")
  for batch in tqdm(skipgram_loader):
    x, y = batch
    x, y = x.to(device), y.to(device) # (B, W), (B)
    output = skipgram(x)  # (B, V)

    optim.zero_grad()
    loss = loss_function(output, y)
    loss.backward()
    optim.step()

    print(f"Train loss: {loss.item()}")

print("Finished.")

  0%|          | 0/64 [00:00<?, ?it/s]

##################################################
Epoch: 1
Train loss: 4.416653633117676
Train loss: 4.013772010803223
Train loss: 4.249093055725098
Train loss: 4.1817946434021
Train loss: 4.208499908447266
Train loss: 4.35623836517334
Train loss: 4.150032997131348
Train loss: 4.082703590393066
Train loss: 4.06230354309082
Train loss: 4.322841644287109
Train loss: 4.178714275360107
Train loss: 4.357064723968506
Train loss: 4.002449989318848
Train loss: 4.264979839324951
Train loss: 3.8697385787963867
Train loss: 4.212023735046387
Train loss: 4.058620929718018
Train loss: 4.153619766235352
Train loss: 4.097601413726807
Train loss: 4.269953727722168
Train loss: 3.69059681892395
Train loss: 4.287111282348633
Train loss: 4.496732711791992
Train loss: 3.958530902862549
Train loss: 4.1485185623168945
Train loss: 3.9942409992218018
Train loss: 4.996145248413086
Train loss: 4.622960567474365
Train loss: 4.168692588806152
Train loss: 4.0700273513793945
Train loss: 4.173990249633789
Train loss:

  0%|          | 0/64 [00:00<?, ?it/s]

Train loss: 3.8217015266418457
Train loss: 4.264951229095459
Train loss: 4.341148853302002
Train loss: 4.223560333251953
Train loss: 4.4063520431518555
Train loss: 3.756385087966919
Train loss: 4.123242378234863
Train loss: 4.69522762298584
Train loss: 4.038976192474365
Train loss: 4.231125354766846
Train loss: 4.075807571411133
Train loss: 4.414292335510254
Train loss: 4.181360721588135
Train loss: 4.457058429718018
Train loss: 4.227865695953369
Train loss: 4.317999362945557
Train loss: 4.011598587036133
Train loss: 3.9067890644073486
Train loss: 4.626102447509766
Train loss: 4.0340352058410645
Train loss: 3.934690475463867
Train loss: 3.9926276206970215
Train loss: 4.305391788482666
Train loss: 4.349721908569336
##################################################
Epoch: 2
Train loss: 4.384714126586914
Train loss: 3.9742300510406494
Train loss: 4.215574741363525
Train loss: 4.132747650146484
Train loss: 4.181686878204346
Train loss: 4.330953121185303
Train loss: 4.114665985107422
Train

  0%|          | 0/64 [00:00<?, ?it/s]

Train loss: 4.56103515625
Train loss: 3.963407278060913
Train loss: 3.884793281555176
Train loss: 3.928325653076172
Train loss: 4.251493453979492
Train loss: 4.263535499572754
##################################################
Epoch: 4
Train loss: 4.3225555419921875
Train loss: 3.8958168029785156
Train loss: 4.1490960121154785
Train loss: 4.037614822387695
Train loss: 4.128446102142334
Train loss: 4.281099319458008
Train loss: 4.0456366539001465
Train loss: 4.0063042640686035
Train loss: 3.9700677394866943
Train loss: 4.218389987945557
Train loss: 4.09381628036499
Train loss: 4.25820255279541
Train loss: 3.9470126628875732
Train loss: 4.185772895812988
Train loss: 3.7977542877197266
Train loss: 4.118760108947754
Train loss: 3.9723691940307617
Train loss: 4.063939571380615
Train loss: 4.013425827026367
Train loss: 4.1676249504089355
Train loss: 3.3963818550109863
Train loss: 4.048434257507324
Train loss: 4.3782196044921875
Train loss: 3.873351573944092
Train loss: 4.033586502075195
Trai

100%|██████████| 64/64 [00:00<00:00, 762.38it/s]

Train loss: 4.076918601989746
Train loss: 4.104693412780762
Train loss: 3.821932792663574
Train loss: 4.175565719604492
Train loss: 3.706104278564453
Train loss: 4.113001823425293
Train loss: 4.1017889976501465
Train loss: 3.975160837173462
Train loss: 3.9692611694335938
Train loss: 3.381242036819458
Train loss: 3.8141989707946777
Train loss: 4.533492088317871
Train loss: 3.936282157897949
Train loss: 4.114580154418945
Train loss: 3.9723989963531494
Train loss: 4.223391532897949
Train loss: 4.010232925415039
Train loss: 4.338778495788574
Train loss: 4.099018096923828
Train loss: 4.211714744567871
Train loss: 3.8315045833587646
Train loss: 3.8119611740112305
Train loss: 4.497283458709717
Train loss: 3.8938217163085938
Train loss: 3.8353941440582275
Train loss: 3.865853786468506
Train loss: 4.199726104736328
Train loss: 4.1787004470825195
Finished.


### **테스트**

학습된 각 모델을 이용하여 test 단어들의 word embedding을 확인.

In [20]:
for word in test_words:
  input_id = torch.LongTensor([w2i[word]]).to(device)
  emb = cbow.embedding(input_id)

  print(f"Word: {word}")
  print(emb.squeeze(0))

Word: 음식
tensor([-0.4879, -0.4811,  0.8295, -0.0576, -2.7324,  0.8201, -0.7498, -0.0534,
         1.7605,  1.0936,  0.5006,  1.9304, -1.8520, -0.7684,  0.0975, -0.4225,
        -1.1209,  0.3386,  1.1958,  1.4164, -0.0192,  1.8523, -1.9027,  1.1389,
        -1.3449,  0.8548, -1.1357,  0.9903, -1.8901,  0.8813, -0.2777,  1.1343,
         0.3698,  0.9501,  0.4275, -0.1072,  0.4278, -2.1341,  0.0475,  2.6164,
         0.5634, -1.8851,  0.2864,  1.1148,  0.5337, -1.5166,  1.7710,  0.8982,
        -1.1035,  0.5978,  0.1972,  0.0300,  0.5883,  0.7569,  0.4732,  0.3068,
        -0.6523,  0.2161,  0.0133,  1.4295,  0.3007, -0.9130,  0.1234, -1.1884,
        -0.3102, -0.3408, -0.5070, -0.8260, -0.3256,  1.3012, -0.1230,  0.1782,
        -0.1277, -0.6313,  1.2273, -0.6418, -0.6517,  1.0611,  0.6232, -0.8748,
         0.4840,  1.7349,  1.0964, -0.4120, -0.8949,  0.4366, -1.3554,  0.4949,
         0.1012,  0.5599,  1.0188,  0.7567,  1.3228,  1.5533,  0.3783,  0.1646,
         1.9877,  0.0946, -0.00

In [21]:
for word in test_words:
  input_id = torch.LongTensor([w2i[word]]).to(device)
  emb = skipgram.embedding(input_id)

  print(f"Word: {word}")
  print(max(emb.squeeze(0)))

Word: 음식
tensor(3.5106, device='cuda:0', grad_fn=<UnbindBackward>)
Word: 맛
tensor(2.7780, device='cuda:0', grad_fn=<UnbindBackward>)
Word: 서비스
tensor(3.4424, device='cuda:0', grad_fn=<UnbindBackward>)
Word: 위생
tensor(2.7496, device='cuda:0', grad_fn=<UnbindBackward>)
Word: 가격
tensor(2.7950, device='cuda:0', grad_fn=<UnbindBackward>)
